# YOLOv5 Video Object Detection

### Project Overview
This notebook demonstrates how to perform object detection on a video file using the **YOLOv5** model. It processes a video frame by frame, detects objects in each frame, and saves the results as a new annotated video file. This project showcases proficiency in real-time object detection and video processing, which are crucial skills for computer vision applications.

### Methodology
1.  **Setup and Model Loading:** The notebook installs the necessary libraries and loads a pre-trained YOLOv5 model from PyTorch Hub.
2.  **Video Processing:** It accesses a video file from Google Drive, reads it frame by frame using `OpenCV`, and runs the YOLOv5 model on each frame.
3.  **Visualization:** It draws bounding boxes and labels on each frame to visualize the detected objects.
4.  **Output Generation:** The processed frames are compiled and saved as a new video file in the user's Google Drive, making it easy to download and share.

### Technologies Used
- Python
- PyTorch
- OpenCV
- YOLOv5

In [ ]:
# --- Section 1: Setup and Model Loading ---

import torch
import cv2
import numpy as np
from google.colab import drive

# Install the required library for YOLOv5
try:
    import ultralytics
except ImportError:
    print("Installing ultralytics library...")
    !pip install -U 'ultralytics'
    import ultralytics

print("Loading pre-trained YOLOv5 model from PyTorch Hub...")
try:
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading YOLOv5 model: {e}")
    raise # Re-raise the exception

# Set model confidence and NMS thresholds
model.conf = 0.4  # Set confidence threshold to 40%
model.iou = 0.5   # Set IoU threshold for NMS to 50%

# --- Section 2: Mount Google Drive and Specify Video Paths ---

print("\nMounting Google Drive to access video file...")
drive.mount('/content/drive')

# IMPORTANT: Replace with the actual path to your video file in Google Drive
video_path = '/content/drive/MyDrive/path_to_your_video.mp4'

# IMPORTANT: Replace with the desired output path for the processed video
output_video_path = '/content/drive/MyDrive/yolov5_output_video.mp4'

print(f"\nInput video path: {video_path}")
print(f"Output video path: {output_video_path}")

# --- Section 3: Video Object Detection ---

print("\nStarting video processing...")

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video file.")
else:
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Define the codec and create VideoWriter object to save the output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # You can use other codecs like 'XVID'
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Perform inference
        results = model(frame)
        
        # Render bounding boxes and labels on the frame
        results.render()
        annotated_frame = results.ims[0]
        
        # Write the annotated frame to the output video
        out.write(cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR))
        
        frame_count += 1
        print(f"Processed frame: {frame_count}", end='\r')

    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print("\n\nVideo processing complete. The output video is saved to your Google Drive.")